In [1]:
import numpy as np

In [2]:
a = [1,2,3,4,5]
b = ['-', '*', "/", '+']
muChance = 0.05











In [3]:
def generatePoulation(a,b,n):
    aCopy = a.copy()
    pop = []
    operandPop = []
    for i in range(n):
        operandChrom = []
        lenb = len(b)
        for j in range(lenb):
            if j != 0:
                bb = np.random.randint(len(b))
                operandChrom.append(b[bb])

        operandPop.append(operandChrom)
        chrom = []
        lena = len(a)
        for j in range(lena):
            aa = np.random.randint(len(a))
            chrom.append(a[aa])
            del a[aa]
        pop.append(chrom)
        a = aCopy.copy()
    return pop, operandPop

In [4]:
def operandMutation(operand, muChance, b):
    for i in range(len(operand)):
        if np.random.random()>=float(1-muChance):
            operand[i] = b[np.random.randint(len(b))]
    return operand

In [8]:
def ChromeMutation(chrome, muChance):
    for i in range(len(chrome)):
        if np.random.random() >= float(1-muChance):
            g1 = np.random.randint(len(chrome))
            g2 = np.random.randint(len(chrome))
            buf = chrome[g2]
            chrome[g2] = chrome[g1]
            chrome[g1] = buf
    return chrome

In [5]:
pop, popOperand = generatePoulation(a,b,3)

In [6]:
print(pop, popOperand)
operandMutation(popOperand[1], muChance, b)

[[1, 3, 4, 2, 5], [2, 3, 4, 5, 1], [4, 1, 2, 3, 5]] [['-', '-', '-'], ['+', '/', '-'], ['*', '/', '+']]


['/', '/', '-']

In [77]:
ChromeMutation(pop[2], muChance)

[2, 5, 3, 1, 4]